In [1]:
import numpy as np
from tabulate import tabulate

In [2]:
A = np.array([[2,-1,6],[0,1,-1],[-2,2,1]])

Todo: 
 * Determine bounds (need another resource)
 * Make results into data frame?
 * Work out where this is parallelizable? 
     * Strategy iteration and value calculation are separate
 * Work out what memory doesn't have to be saved; e.g. upper and lower value don't actually matter?
     * We need to keep
         * The payoff matrix
         * The current *s*
         * The strategy list
         * The current upper/lower value
 

In [44]:
class naive_fp():

    def __init__(self, 
                 payoff_matrix, 
                 initial_i=0,
                 print_head=5,
                 print_tail=5,
                 print_indexing_at_1 = False):
        """Variables named according to Ferguson's 'Game Theory',
        part II, chapter 4, page 46.
        
        Note: to ease indexing, the strategies begin at 0; i.e.,
        a 3x3 matrix will have strategies 0, 1, and 2 rather than
        1, 2, and 3. This also means that the index for the `round`
        starts at k = 0 (meaning only the inital play has occured).
        """
        
        # Set how many lines to pretty-print
        self.print_head, self.print_tail = print_head, print_tail 
        # Set whether the indexing for printing starts a 0 (default) or 1
        # (makes it easier to see if the values match the values in texts that
        # state at k = 1)
        self.print_indexing_at_1 = print_indexing_at_1
        
        
        self.payoff_matrix = payoff_matrix
        
        # Round
        self.k = 0
        
        # Pure strategy selections for player I
        self.i = np.array([initial_i], dtype = int)
        # Incremental payoffs for player II
        self.s = np.array([self.payoff_matrix[self.i[self.k]]])
     
        # Pure strategy selection for player II
        self.j = np.array([np.argmin(self.s[self.k])], dtype = int)
        # Incremental payoffs for player I
        self.t = np.array([self.payoff_matrix[:, self.j[self.k]]])
        
        # Select best-response for player I; choose i_{k+1}
        self.i = np.append(self.i, np.argmax(self.t[self.k]))
        
        # Game value lists
        self.v_upper = np.array([self.t[self.k][self.i[self.k + 1]]], dtype = np.float64)
        self.v_lower = np.array([self.s[self.k][self.j[self.k]]], dtype = np.float64)
        
        # supremum of v_upper
        self.upper_bound = self.v_upper
        # infimum of v_lower
        self.lower_bound = self.v_lower
        
        ## Best strategies on round k
        # Player I's initial strategy is the first row with probability 1
        self.player_I_optimal_strategy = np.zeros([payoff_matrix.shape[0]])
        self.player_I_optimal_strategy[0] = 1
        # Player II's initial strategy is j[0] with probability 1
        self.player_II_optimal_strategy = np.zeros([payoff_matrix.shape[1]])
        self.player_II_optimal_strategy[self.j[0]] = 1
        
        # Increment round 
        self.k += 1
        
    def next_round(self, rounds=1):
        """The rounds are calculated as follows:
        At the end of each round, all values of i[k], s[k], j[k], t[k], v_upper[k] and v_lower[k]
        have been updated.
        
        During each round, they are updated in that order.
        """

        
        for _ in range(rounds):        
            # Increment payoffs for player II; calculate s_k
            self.s = np.append(self.s, [self.s[self.k - 1] + self.payoff_matrix[self.i[self.k]]], axis=0)
            # Select best-response for player II; choose j_k
            self.j = np.append(self.j, np.argmin(self.s[self.k]))
        
            # Increment payoffs for player I: calculate t_k
            self.t = np.append(self.t, [self.t[self.k - 1] + self.payoff_matrix[:, self.j[self.k]]], axis = 0)
            # Select best-response for player I; choose i_k
            self.i = np.append(self.i, np.argmax(self.t[self.k]))
       
            ## Change value bounds
            # Calculate v_upper_{k-1}
            self.v_upper = np.append(self.v_upper, 1 / (self.k+1) * self.t[self.k][self.i[self.k+1]])
            # Calculate v_lower_k
            self.v_lower = np.append(self.v_lower, 1 / (self.k + 1) * self.s[self.k][self.j[self.k]])
            
            #Increment round
            self.k += 1
        
        # It would be faster to not have to compare over the whole list, but this is the naive 
        # implementation
        self.upper_bound = {'value' : min(self.v_upper), 'index' : np.argmin(self.v_upper)}
        self.lower_bound = {'value' : max(self.v_lower), 'index' : np.argmax(self.v_lower)}
      
        ## Update Optimal strategies    
        # Zero out the strategies
        self.player_I_optimal_strategy = np.zeros(self.payoff_matrix.shape[0])
        # Get the number of times each pure strategy was played up to the inf
        strats_I, counts_I = np.unique(self.i[: self.lower_bound['index'] + 1], return_counts = True)
        # Play each strategy equally likely
        for i,j in zip(strats_I, counts_I):
            self.player_I_optimal_strategy[i] = j / (self.lower_bound['index']+1)
    
        # Same for player II
        self.player_II_optimal_strategy = np.zeros(self.payoff_matrix.shape[1])
        # Get the number of times each pure strategy was played up to the sup
        strats_II, counts_II = np.unique(self.j[: self.upper_bound['index'] + 1], return_counts = True)
        for i,j in zip(strats_II, counts_II):
            self.player_II_optimal_strategy[i] = j / (self.upper_bound['index']+1)   
    
    
    def __str__(self):
        ph = self.print_head
        pt = self.print_tail
        
        headers = ('k', 'i_k', 's_k', 'v_lower_k', 'j_k', 't_k', 'v_upper_k')
        
        bounds_str = "\n\nUpper game value: " + str(self.upper_bound) + "\n" \
            "Lower game value: " + str(self.lower_bound) + "\n\n"
        
        strategies_str = "Player I optimal strategy: \n\t" + str(list(self.player_I_optimal_strategy)) + \
                         "\nPlayer II optimal strategy: \n\t" + str(list(self.player_II_optimal_strategy)) +\
                         "\n\n"
        
        # Build up a return value as we go
        rv = bounds_str + strategies_str
        
        if ph + pt >= self.k+1:
            # If the total number of lines to print is less than or equal to the
            # number of rounds, we don't need to print a line of '[...]'.
            
            # Return table with inidces starting at 1
            if self.print_indexing_at_1 == True:
                data = zip(
                    list(range(1, self.k + 1)), 
                    self.i + 1, 
                    self.s, 
                    self.v_lower, 
                    self.j + 1, 
                    self.t, 
                    self.v_upper)
                rv += tabulate(data, headers = headers)
                return rv
                
            # Return table with indices starting at 0
            data = zip(
                list(range(self.k + 1)), 
                self.i, 
                self.s, 
                self.v_lower, 
                self.j, 
                self.t, 
                self.v_upper)
            rv += tabulate(data, headers = headers)
            return rv
        
        # Total number of rounds exceeds ph+pt; print a line break
        line_break = "\n\n[...]\n" + str(self.k - ph - pt) + " lines skipped... \n" + "[...]\n\n"
        
        if self.print_indexing_at_1 == True:
                data_head = list(zip(
                    list(range(1, ph + 1)), 
                    self.i[:ph] + 1, 
                    self.s[:ph], 
                    self.v_lower[:ph], 
                    self.j[:ph] + 1, 
                    self.t[:ph], 
                    self.v_upper[:ph]))
                 
                rv += tabulate(data_head, headers = headers)
                rv += line_break
                
                data_tail = list(zip(
                    list(range(self.k - pt + 1, self.k + 2)), 
                    self.i[self.k - pt:self.k] + 1, 
                    self.s[self.k - pt:self.k], 
                    self.v_lower[self.k - pt:self.k], 
                    self.j[self.k - pt:self.k] + 1, 
                    self.t[self.k - pt:self.k], 
                    self.v_upper[self.k - pt:self.k]))
                
                rv += tabulate(data_tail, headers = headers)
                
                return rv
            
        # Return table with indices starting at 0
        data_head = list(zip(
                    list(range(ph)), 
                    self.i[:ph], 
                    self.s[:ph], 
                    self.v_lower[:ph], 
                    self.j[:ph], 
                    self.t[:ph], 
                    self.v_upper[:ph]))
        
        rv += tabulate(data_head, headers = headers)            
        rv += line_break
        
        data_tail = list(zip(
                    list(range(self.k  - pt, self.k + 2)), 
                    self.i[self.k - pt:self.k], 
                    self.s[self.k - pt:self.k], 
                    self.v_lower[self.k - pt:self.k], 
                    self.j[self.k - pt:self.k], 
                    self.t[self.k - pt:self.k], 
                    self.v_upper[self.k - pt:self.k]))
        
        rv += tabulate(data_tail, headers = headers)
        return rv

In [45]:
A_fp = naive_fp(A)

In [46]:
A_fp.next_round(90)


In [47]:
print(A_fp)



Upper game value: {'value': 0.5, 'index': 1}
Lower game value: {'value': 0.4835164835164836, 'index': 90}

Player I optimal strategy: 
	[0.27472527472527475, 0.6923076923076923, 0.03296703296703297]
Player II optimal strategy: 
	[0.5, 0.5, 0.0]

  k    i_k  s_k           v_lower_k    j_k  t_k           v_upper_k
---  -----  ----------  -----------  -----  ----------  -----------
  0      0  [ 2 -1  6]        -1         1  [-1  1  2]     2
  1      2  [0 1 7]            0         0  [1 1 0]        0.5
  2      0  [ 2  0 13]         0         1  [0 2 2]        0.666667
  3      1  [ 2  1 12]         0.25      1  [-1  3  4]     1
  4      2  [ 0  3 13]         0         0  [1 3 2]        0.6

[...]
81 lines skipped... 
[...]

  k    i_k  s_k           v_lower_k    j_k  t_k           v_upper_k
---  -----  ----------  -----------  -----  ----------  -----------
 86      1  [44 40 94]     0.45977       1  [36 46 10]     0.528736
 87      1  [44 41 93]     0.465909      1  [35 47 12]     0.

In [48]:
B_fp = naive_fp(np.array([[4,1,2,3,4,5],[-1,3,5,-2,-5,0]]))

In [51]:
B_fp.next_round(100)

In [52]:
print(B_fp)



Upper game value: {'value': 1.5454545454545454, 'index': 10}
Lower game value: {'value': 1.5454545454545456, 'index': 98}

Player I optimal strategy: 
	[0.7272727272727273, 0.2727272727272727]
Player II optimal strategy: 
	[0.0, 0.8181818181818182, 0.0, 0.0, 0.18181818181818182, 0.0]

  k    i_k  s_k                    v_lower_k    j_k  t_k        v_upper_k
---  -----  -------------------  -----------  -----  -------  -----------
  0      0  [4 1 2 3 4 5]                1        1  [1 3]            3
  1      1  [ 3  4  7  1 -1  5]         -0.5      4  [ 5 -2]          2.5
  2      0  [ 7  5  9  4  3 10]          1        4  [ 9 -7]          3
  3      0  [11  6 11  7  7 15]          1.5      1  [10 -4]          2.5
  4      0  [15  7 13 10 11 20]          1.4      1  [11 -1]          2.2

[...]
92 lines skipped... 
[...]

  k    i_k  s_k                          v_lower_k    j_k  t_k          v_upper_k
---  -----  -------------------------  -----------  -----  ---------  -----------

In [59]:
C = np.array([[0,-1,1],[2,0,-2],[-3,3,0]])

In [60]:
C_fp = naive_fp(C)

In [61]:
print(C_fp)



Upper game value: [3.]
Lower game value: [-1.]

Player I optimal strategy: 
	[1.0, 0.0, 0.0]
Player II optimal strategy: 
	[0.0, 1.0, 0.0]

  k    i_k  s_k           v_lower_k    j_k  t_k           v_upper_k
---  -----  ----------  -----------  -----  ----------  -----------
  0      0  [ 0 -1  1]           -1      1  [-1  0  3]            3


In [62]:
C_fp.next_round(100)

In [63]:
print(C_fp)



Upper game value: {'value': 0.0, 'index': 20}
Lower game value: {'value': 0.0, 'index': 10}

Player I optimal strategy: 
	[0.5454545454545454, 0.2727272727272727, 0.18181818181818182]
Player II optimal strategy: 
	[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]

  k    i_k  s_k           v_lower_k    j_k  t_k           v_upper_k
---  -----  ----------  -----------  -----  ----------  -----------
  0      0  [ 0 -1  1]    -1             1  [-1  0  3]      3
  1      2  [-3  2  1]    -1.5           0  [-1  2  0]      1
  2      1  [-1  2 -1]    -0.333333      0  [-1  4 -3]      1.33333
  3      1  [ 1  2 -3]    -0.75          2  [ 0  2 -3]      0.5
  4      1  [ 3  2 -5]    -1             2  [ 1  0 -3]      0.2

[...]
91 lines skipped... 
[...]

  k    i_k  s_k              v_lower_k    j_k  t_k           v_upper_k
---  -----  -------------  -----------  -----  ----------  -----------
 96      1  [ 14  -2 -12]    -0.123711      2  [ 0  2 -3]    0.0206186
 97      1  [ 16  

In [64]:
(6/11,3/11,2/11)

(0.5454545454545454, 0.2727272727272727, 0.18181818181818182)

In [115]:
R = np.random.rand(100,100)

In [116]:
R_fp = naive_fp(R)

In [120]:
%time R_fp.next_round(1000)

CPU times: user 737 ms, sys: 9 µs, total: 737 ms
Wall time: 736 ms


Improvements:
 * On each calculation of $s$, we're going to calculate the new $inf(v)$ immediately
 * 
 

In [ ]:
## Second pass

class naive_fp_2():

    def __init__(self, 
                 payoff_matrix, 
                 initial_i=0):
        """Variables named according to Ferguson's 'Game Theory',
        part II, chapter 4, page 46.
        
        Note: to ease indexing, the strategies begin at 0; i.e.,
        a 3x3 matrix will have strategies 0, 1, and 2 rather than
        1, 2, and 3. This also means that the index for the `round`
        starts at k = 0 (meaning only the inital play has occured).
        """
                
        self.payoff_matrix = payoff_matrix
        
        # Round
        self.k = 0
        
        # Pure strategy selections for player I; we only keep the ones we need
        self.i_buff = np.array([initial_i], dtype = int)
 
        # Incremental payoffs for player II
        self.s_last = [self.payoff_matrix[self.i[self.k]]]
     
        # Pure strategy selection for player II
        self.j_buff = np.array([np.argmin(self.s[self.k])], dtype = int)
        # Incremental payoffs for player I
        self.t_last = np.array([self.payoff_matrix[:, self.j[self.k]]])
        
        # Select best-response for player I; choose i_{k+1}
        self.i_buff = np.append(self.i, np.argmax(self.t[self.k]))
        
        self.sup_v_round_tuple = (np.array([self.t[self.k][self.i[self.k + 1]]], dtype = np.float64), 0)
        self.inf_v_round_tuple = (np.array([self.s[self.k][self.j[self.k]]], dtype = np.float64), 0)
        
        ## Best strategies on round k
        # Player I's initial strategy is the first row with probability 1
        self.player_I_optimal_strategy = np.zeros([payoff_matrix.shape[0]])
        self.player_I_optimal_strategy[0] = 1
        # Player II's initial strategy is j[0] with probability 1
        self.player_II_optimal_strategy = np.zeros([payoff_matrix.shape[1]])
        self.player_II_optimal_strategy[self.j_buff[0]] = 1
        
        # Increment round 
        self.k += 1
        
    def next_round(self, rounds=1):
        for _ in range(rounds):        
            # Increment payoffs for player II; calculate s_k
            s_k = self.s_last + self.payoff_matrix[self.i_buff[self.k]]
            # Select best-response for player II; choose j_k
            self.j_buff = np.append(self.j, np.argmin(s_k)
            
            # Calculate v_lower
            v_lower = 1 / (self.k + 1) * s_k[self.j_buff[self.k]])
            
            if v_lower < self.lower_bound
            
             # Calculate v_upper_{k-1}
            self.v_upper = np.append(self.v_upper, 1 / (self.k+1) * self.t[self.k][self.i[self.k+1]])
            
            
            
            
            
            
            
        
            # Increment payoffs for player I: calculate t_k
            self.t = np.append(self.t, [self.t[self.k - 1] + self.payoff_matrix[:, self.j[self.k]]], axis = 0)
            # Select best-response for player I; choose i_k
            self.i = np.append(self.i, np.argmax(self.t[self.k]))
       
            ## Change value bounds
           
            
            
            #Increment round
            self.k += 1
        
        # It would be faster to not have to compare over the whole list, but this is the naive 
        # implementation
        self.upper_bound = {'value' : min(self.v_upper), 'index' : np.argmin(self.v_upper)}
        self.lower_bound = {'value' : max(self.v_lower), 'index' : np.argmax(self.v_lower)}
      
        ## Update Optimal strategies    
        # Zero out the strategies
        self.player_I_optimal_strategy = np.zeros(self.payoff_matrix.shape[0])
        # Get the number of times each pure strategy was played up to the inf
        strats_I, counts_I = np.unique(self.i[: self.lower_bound['index'] + 1], return_counts = True)
        # Play each strategy equally likely
        for i,j in zip(strats_I, counts_I):
            self.player_I_optimal_strategy[i] = j / (self.lower_bound['index']+1)
    
        # Same for player II
        self.player_II_optimal_strategy = np.zeros(self.payoff_matrix.shape[1])
        # Get the number of times each pure strategy was played up to the sup
        strats_II, counts_II = np.unique(self.j[: self.upper_bound['index'] + 1], return_counts = True)
        for i,j in zip(strats_II, counts_II):
            self.player_II_optimal_strategy[i] = j / (self.upper_bound['index']+1)   
    
    
    def __str__(self):
        ph = self.print_head
        pt = self.print_tail
        
        headers = ('k', 'i_k', 's_k', 'v_lower_k', 'j_k', 't_k', 'v_upper_k')
        
        bounds_str = "\n\nUpper game value: " + str(self.upper_bound) + "\n" \
            "Lower game value: " + str(self.lower_bound) + "\n\n"
        
        strategies_str = "Player I optimal strategy: \n\t" + str(list(self.player_I_optimal_strategy)) + \
                         "\nPlayer II optimal strategy: \n\t" + str(list(self.player_II_optimal_strategy)) +\
                         "\n\n"
        
        # Build up a return value as we go
        rv = bounds_str + strategies_str
        
        if ph + pt >= self.k+1:
            # If the total number of lines to print is less than or equal to the
            # number of rounds, we don't need to print a line of '[...]'.
            
            # Return table with inidces starting at 1
            if self.print_indexing_at_1 == True:
                data = zip(
                    list(range(1, self.k + 1)), 
                    self.i + 1, 
                    self.s, 
                    self.v_lower, 
                    self.j + 1, 
                    self.t, 
                    self.v_upper)
                rv += tabulate(data, headers = headers)
                return rv
                
            # Return table with indices starting at 0
            data = zip(
                list(range(self.k + 1)), 
                self.i, 
                self.s, 
                self.v_lower, 
                self.j, 
                self.t, 
                self.v_upper)
            rv += tabulate(data, headers = headers)
            return rv
        
        # Total number of rounds exceeds ph+pt; print a line break
        line_break = "\n\n[...]\n" + str(self.k - ph - pt) + " lines skipped... \n" + "[...]\n\n"
        
        if self.print_indexing_at_1 == True:
                data_head = list(zip(
                    list(range(1, ph + 1)), 
                    self.i[:ph] + 1, 
                    self.s[:ph], 
                    self.v_lower[:ph], 
                    self.j[:ph] + 1, 
                    self.t[:ph], 
                    self.v_upper[:ph]))
                 
                rv += tabulate(data_head, headers = headers)
                rv += line_break
                
                data_tail = list(zip(
                    list(range(self.k - pt + 1, self.k + 2)), 
                    self.i[self.k - pt:self.k] + 1, 
                    self.s[self.k - pt:self.k], 
                    self.v_lower[self.k - pt:self.k], 
                    self.j[self.k - pt:self.k] + 1, 
                    self.t[self.k - pt:self.k], 
                    self.v_upper[self.k - pt:self.k]))
                
                rv += tabulate(data_tail, headers = headers)
                
                return rv
            
        # Return table with indices starting at 0
        data_head = list(zip(
                    list(range(ph)), 
                    self.i[:ph], 
                    self.s[:ph], 
                    self.v_lower[:ph], 
                    self.j[:ph], 
                    self.t[:ph], 
                    self.v_upper[:ph]))
        
        rv += tabulate(data_head, headers = headers)            
        rv += line_break
        
        data_tail = list(zip(
                    list(range(self.k  - pt, self.k + 2)), 
                    self.i[self.k - pt:self.k], 
                    self.s[self.k - pt:self.k], 
                    self.v_lower[self.k - pt:self.k], 
                    self.j[self.k - pt:self.k], 
                    self.t[self.k - pt:self.k], 
                    self.v_upper[self.k - pt:self.k]))
        
        rv += tabulate(data_tail, headers = headers)
        return rv